In [2]:
from hjst_model.config import DatasetKVSConfig, LayerModelConfig
from hjst_model import layers
import numpy as np
import scipy
import nmslib
import pandas as pd
import os

#カラム内の文字数。デフォルトは50だった
pd.set_option("display.max_colwidth", 800)

#行数
pd.set_option("display.max_rows", 101)

In [3]:
dataset_name = "tokai"
CONF_DIR = './configs{}'.format('')
MODEL_DIR = './results/hjst{}/models/'.format('')
layer_conf = LayerModelConfig()
layer_conf.link(os.path.join(CONF_DIR, './layer.conf'), create_if_missing=True)
layer_conf.set_directory(MODEL_DIR, os.path.join(CONF_DIR, 'dataset.conf'))
layer_conf.update_file()

for l in ['Sentence']:
    print('train', l)
    layer_conf.create_layer(dataset_name, layers.TfidfLayer, l, 'tfidf_tokai_spm_8000', tokenizer="spm", vocab_size=8000, lsi_size=500, idf=True)

train Sentence


In [ ]:
for l in ['Law', 'Article', 'Sentence']:
    print('train', l)
    layer_conf.create_layer(dataset_name, layers.SWEMMaxLayer, l, 'swem_concat_tokai_spm_16000', tokenizer="spm", vocab_size=16000)

In [4]:
l = layer_conf.load_model("swem_max_am_spm_16000-Sentence")

In [ ]:
TESTSET = "all"

testset_conf = DatasetKVSConfig()
testset_conf.link("./configs/test/dataset.conf")
testset_conf.change_section(TESTSET)
testset = testset_conf.prepare_dataset()

In [ ]:
# prepare layer config instance
layerconf = LayerModelConfig()
layerconf.link("./configs/layer.conf")

In [ ]:
# load sentence layer
layerconf.change_section("wva_tokai_spm_16000-Sentence")
sentence_layer = layerconf.load_model()

In [ ]:
# load law layer
layerconf.change_section("wva_tokai_spm_16000-Law")
law_layer = layerconf.load_model()

In [ ]:
testset.set_iterator_mode(level="Law", tag=True, sentence=False)
law_tags = [tag for tag in testset]
law_vectors = np.matrix([law_layer[tag] for tag in law_tags])
print(law_vectors.shape)

# initialize a new index, using a HNSW index on Cosine Similarity
law_index = nmslib.init(method='hnsw', space='cosinesimil')
law_index.addDataPointBatch(law_vectors)
law_index.createIndex({'post': 2}, print_progress=True)

law_neighbours = law_index.knnQueryBatch(law_vectors, k=10, num_threads=4)

In [ ]:
testset.set_iterator_mode(level="Sentence", tag=True, sentence=False)
sentence_tags = [tag for tag in testset]
sentence_vectors = np.matrix([sentence_layer[tag] for tag in sentence_tags])
print(sentence_vectors.shape)

# initialize a new index, using a HNSW index on Cosine Similarity
sentence_index = nmslib.init(method='hnsw', space='cosinesimil')
sentence_index.addDataPointBatch(sentence_vectors)
sentence_index.createIndex({'post': 2}, print_progress=True)

sentence_neighbours = sentence_index.knnQueryBatch(sentence_vectors, k=10, num_threads=4)

In [ ]:
gen = ([ids, distances] for ids, distances in law_neighbours)

In [ ]:
ltag_gen = (ltag for ltag in law_tags)

In [ ]:
# setting
k = 3

# get query law tag
query = next(ltag_gen)

# get similar law tags and vectors(= simple method output)
target_law_indices, _ = law_index.knnQuery(law_layer[query], k=k+1)
target_law_tags = [law_tags[i] for i in target_law_indices if law_tags[i] != query][:k]

# construct target sentence space
target_spaces = dict()
target_sentence_tags = dict()
for ltag in target_law_tags:
    testset.set_iterator_mode(level="Sentence", tag=True, sentence=False)
    target_sentence_tags[ltag] = [stag for atag in testset['edges']['Law'][ltag] for stag in testset['edges']['Article'][atag]]
    target_sentence_vectors = np.matrix([sentence_layer[tag] for tag in target_sentence_tags[ltag]])

    # initialize a new index, using a HNSW index on Cosine Similarity
    target_spaces[ltag] = nmslib.init(method='hnsw', space='cosinesimil')
    target_spaces[ltag].addDataPointBatch(target_sentence_vectors)
    target_spaces[ltag].createIndex({'post': 2}, print_progress=True)

columns = [str(testset.kvsdicts["root"][os.path.split(tag)[0]])+"({})".format(os.path.split(tag)[0]) for tag in [query]+target_law_tags]
df = pd.DataFrame(columns=columns)
sid = 0
for a in testset.kvsdicts["edges"]["Law"][query]:
    for j, s in enumerate(testset.kvsdicts["edges"]["Article"][a]):
        sid += 1
        df.loc[sid, columns[0]] = testset.kvsdicts["texts"]["Sentence"][s]
        for l, ltag in enumerate(target_law_tags):
            index = target_spaces[ltag]
            resi, resd  = index.knnQuery(sentence_layer[s], 1)
            df.loc[sid, columns[l+1]] = testset.kvsdicts["texts"]["Sentence"][target_sentence_tags[ltag][resi[0]]]
df

In [ ]:
sid = 0
for a in testset.kvsdicts["edges"]["Law"][query]:
    for t, s in enumerate(testset.kvsdicts["edges"]["Article"][a]):
        sid += 1
        print(sid, s)#, testset.kvsdicts["texts"]["Sentence"][s])

In [ ]:
# query for the nearest neighbours of the first datapoint
ids, distances = next(gen)
import os
for i in range(10):
    tag = law_tags[ids[i]]
    print(tag)
    print(i, round(distances[i], 3), testset.kvsdicts["root"][os.path.split(tag)[0]])

In [ ]:
neighbours[0]

In [ ]:
%timeit tree.sparse_distance_matrix(tree, 0.6).toarray()